In [1]:
!pip install gym
!pip install glfw3
!pip install swig
!pip install gym[mujoco]
!pip install pyvirtualdisplay==3.0
!pip install pyglet==1.5.27


ERROR: Could not find a version that satisfies the requirement glfw3 (from versions: none)
ERROR: No matching distribution found for glfw3


In [2]:
!apt-get update
!apt-get install -y xvfb ffmpeg

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

In [13]:
import gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
import moviepy.editor as mpy
from moviepy.editor import VideoFileClip, concatenate_videoclips
from tensorflow.keras.models import load_model
from keras.losses import MeanSquaredError as mse
from pyvirtualdisplay import Display

  and should_run_async(code)



In [4]:
display = Display(visible=0, size=(1400, 900))
display.start()

  and should_run_async(code)



In [5]:
env = gym.make('Humanoid-v4')

  and should_run_async(code)

  deprecation(

  deprecation(



In [6]:
model = Sequential()
model.add(Dense(64, input_dim=env.observation_space.shape[0], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(env.action_space.shape[0], activation='tanh'))

  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



In [7]:
model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

In [8]:
def fitness_function(weights):
    model.set_weights(weights)
    score = 0.0
    for _ in range(5):
        observation = env.reset()
        done = False
        while not done:
            action = model.predict(observation.reshape(1, -1))[0]
            observation, reward, done, info = env.step(action)
            score += reward
    return score

In [9]:
def evolution_strategies(num_generations, population_size, sigma):
    best_score = -np.inf
    best_weights = None
    for generation in range(num_generations):
        # Get the current weights and flatten them
        current_weights = model.get_weights()
        flat_weights = np.concatenate([w.flatten() for w in current_weights])

        # Generate new weights based on the flattened array
        new_flat_weights = np.random.normal(loc=flat_weights, scale=sigma, size=(population_size, flat_weights.shape[0]))

        scores = []
        new_weights_list = [] # Store the reshaped weights for each population member
        for i in range(population_size):
            # Reshape the new weights back to the original structure
            new_weights = []
            start_idx = 0
            for w in current_weights:
                w_size = w.size
                new_w = new_flat_weights[i, start_idx: start_idx + w_size].reshape(w.shape)
                new_weights.append(new_w)
                start_idx += w_size

            new_weights_list.append(new_weights) # Store the reshaped weights
            score = fitness_function(new_weights)
            scores.append(score)

        # Calculate the weighted average for each weight layer separately
        averaged_weights = []
        for i in range(len(current_weights)):
            layer_weights = [w[i] for w in new_weights_list]
            averaged_layer_weights = np.average(layer_weights, axis=0, weights=scores)
            averaged_weights.append(averaged_layer_weights)

        if scores[-1] > best_score:
            best_score = scores[-1]
            best_weights = averaged_weights # Update best_weights with the averaged weights

        sigma *= 0.99
    return best_weights, best_score

In [10]:
def genetic_algorithms(num_generations, population_size, mutation_rate):
    best_score = -np.inf
    best_weights = None
    for generation in range(num_generations):
        # Get the current weights
        current_weights = model.get_weights()

        # Generate new weights for each layer separately
        new_weights = []
        for w in current_weights:
            new_w = np.random.normal(loc=w, scale=1.0, size=w.shape)
            new_weights.append(new_w)

        scores = []
        for _ in range(population_size):
            score = fitness_function(new_weights)
            scores.append(score)

        sorted_indices = np.argsort(scores)[::-1]
        new_weights = [new_weights[i] for i in sorted_indices]  # Sort new_weights based on scores

        for i in range(population_size // 2):
            parent1 = new_weights[i]
            parent2 = new_weights[population_size - i - 1]
            child = []
            for j in range(len(parent1)):
                child_layer = np.zeros_like(parent1[j])
                for k in range(parent1[j].size):
                    if np.random.rand() < mutation_rate:
                        child_layer.flat[k] = np.random.normal(loc=parent1[j].flat[k], scale=0.1)
                    else:
                        child_layer.flat[k] = parent1[j].flat[k]
                child.append(child_layer)
            new_weights[i] = child

        if scores[sorted_indices[0]] > best_score:
            best_score = scores[sorted_indices[0]]
            best_weights = new_weights[0]  # Update best_weights

    return best_weights, best_score

In [11]:
%%capture
es_weights, es_score = evolution_strategies(1, 1, 0.1)
ga_weights, ga_score = genetic_algorithms(1, 1, 0.1)

  if not isinstance(terminated, (bool, np.bool8)):



In [14]:


def generate_video(env, model, filename="humanoid_performance.mp4"):
    frames = []
    observation = env.reset()
    done = False
    while not done:
        frames.append(env.render(mode="rgb_array"))
        action = model.predict(observation.reshape(1, -1))[0]
        observation, reward, done, info = env.step(action)

    env.close()
    clip = mpy.ImageSequenceClip(frames, fps=30)
    clip.write_videofile(filename)

generate_video(env, model)
display.stop()

See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

Moviepy - Done !
Moviepy - video ready humanoid_performance.mp4
